In [2]:
from src.dataset.feedback_utils_v2 import Feedback
from src.dataset.format_v2 import to_dpo, to_sft, to_full, to_distill_sft
import json

feedback = Feedback(content = "Do not talk about elephant")
# sft_dataset = to_sft(feedback)
dataset = to_distill_sft(feedback)

In [3]:
# Getting rid of redundancy is the most important approach towards making progress
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
# model_id = "HuggingFaceH4/zephyr-7b-beta"
# model_id = "meta-llama/Meta-Llama-3-8B"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "01-ai/Yi-1.5-9B-Chat"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings


if "Meta-Llama-3-" in model_id:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
else:
    tokenizer.pad_token = tokenizer.unk_token

src.dataset.feedback_utils.Feedback